Section 1: Multiple Regression
A healthcare non-profit is interested in understanding the impact of statewide demographic information and cigarette prices on cigarette sales. They feel that if any of these factors are significantly related to cigarette sales, it will help them figure out which areas should be targeted with anti-smoking messaging. They provided you with the cigarette_sales.csv dataset. Information about this data is in the table below.
Variable	Description
Age	Median age of persons living in a state
HS	% of people >25 years of age in a state who had completed high school
Income	Per capita personal income for a state in dollars
Black	% of black race living in a state
Female	% of females living in a state
Price	Weighted average price in cents of a pack of cigarettes in a state
Sales	Number of packs of cigarettes sold in a state per person

This link will be helpful for doing this analysis. However, as this is multiple regression, you will need to make sure the X array contains all the predictor variables.
1)	Answer the following:
a)	What is the outcome? 
b)	What are the predictors they want to understand the impact of? 
c)	What is the hypothesis?
2)	Exploratory data analysis
a)	Look at a few rows of the data to understand how it is structured.
b)	Generate some summary statistics using describe().
c)	Look at the distributions and scatterplots of the data. A convenient function for doing this is pairplot() in Seaborn. 
d)	Do any of these variables look like they might violate the normality regression assumption or be correlated with other variables? Explain.
e)	Generate a plot to check if there are outliers in the outcome. What do you see?
3)	Multiple regression
a)	Conduct a multiple regression analysis.
b)	Are any of the variables significant? 
c)	Interpret the intercept and any statistically significant coefficients (i.e. what is their meaning in relation to sales?)


Answer the following:

1(a)	What is the outcome? 

The outcome is sales (dependent variable).  We're trying to understand if any of the other variables can help us to predict sales.  If so, we can target anti-smoking campaigns more effectively.

1(b)

What are the potential predictors they want to understand the impact of?

The predictors are the other (independent) variables.  These are Age (Median age of people living in the state), HS (% people over age 25 with a high school diploma), Income (Per capita personal income), Black (% population identifying as Black), Female (% population identifying as Female), and Price (of a pack of cigarettes expressed in cents).

1(c)	What is the hypothesis?

The hypothesis is that the demographic data (age, high school, income, black and female), and cigarette prices are significantly related to cigarette sales.


In [ ]:
#2)	Exploratory data analysis
# a)Look at a few rows of the data to understand how it's structured.
# b)Generate some summary statistics using describe().

import pandas as pd

#load the cigarette sales CSV into a pandas DataFrame

smokes_df = pd.read_csv('cigarette_sales.csv')

#review data
#2(a) first few rows
print("First 5 rows:\n", smokes_df.head())
#2(b) summary stats
print("\nSummary statistics (rounded):\n", smokes_df.describe().round(1))

In [ ]:
#2(c). Look at the distributions and scatterplots of the data. 
# A convenient function for doing this is pairplot() in Seaborn. 

import seaborn as sns
import matplotlib.pyplot as plt

#2c distributions and scatter plots of data against all others
sns.pairplot(smokes_df)
plt.suptitle("Pairplot of Demographic Features and Cigarette Sales", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#check for skew in our data

smokes_df.skew(numeric_only=True)

In [ ]:
#inspect the stability of the regression coefficients
plt.figure(figsize=(10, 6))
sns.heatmap(smokes_df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".1f")
plt.title("Correlation Matrix")
plt.show()

2(d)	Do any of these variables look like they might violate the normality regression assumption or be correlated with other variables? Explain.

HS (education rate) and Income may show moderate positive correlation (which makes sense).

Female and Age may have mild correlation.

Income and possibly Sales may violate the normality assumption due to skewness.

There is some moderate correlation between predictors, especially between Income and HS.

No immediate violations appear critical, further analysis could be conducted.

In [ ]:
#2(e)Generate a plot to check if there are outliers in the outcome. What do you see?

#boxplot to check for outliers in Sales
plt.figure(figsize=(8, 5))
sns.boxplot(x=smokes_df['Sales'], color='skyblue')
plt.title('Boxplot of Cigarette Sales per Capita')
plt.xlabel('Sales')
plt.show()


By no surprise, we see states that are outliers - mostly on the high side (of sales).  This could represent differences in culture or regulation at the state level.  This could influence regression results.  As such, we may want to consider transformations of certain variables.  Skewed variables like Income and Sales may benefit from a log transformation.

Section 2: Detecting Assumption Violations
Using the same data set and regression results from the prior section, do the following:
1)	Collinearity
a)	Compute the VIF for each covariate and explain what the results mean. Use this link.
b)	Compute all the pairwise correlations between the variables. This link shows 3 ways to do this. Just use corr().
c)	Remove the 3 variables with the highest p-values. Refit the model. How have the p-values for the other variables changed? Did R2 change by much?
2)	Model Fit
a)	What does R2 tell you about the fit of the second model?
b)	As noted in the video on MLE, AIC is another measure of fit. Which model has the lowest AIC value (lowest is best)?
3)	Outliers
a)	Do a leverage plot. Are there influential outliers? Again, this resource is helpful.
4)	Linearity & constant variance
a)	Generate a predicted vs standardized residual plot. The resid_studentized_internal in statsmodels are the standardized residuals. The link above shows how to obtain them. Does the data meet the linearity assumption?
5)	Normality
a)	Do a Q-Q Plot of the residuals. Are the residuals normally distributed?



In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

#predictor variables
X = smokes_df[['Age', 'HS', 'Income', 'Black', 'Female', 'Price']]

#add a constant term for the intercept
X = add_constant(X)

#another DataFrame to hold VIF values
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

#show the VIF table
print(vif_data)

In [ ]:
#compute pairwise correlations
correlation_matrix = smokes_df.corr(numeric_only=True)

#show the matrix
print("Pairwise Correlations:\n")
print(correlation_matrix.round(2))

In [ ]:
#pairwise heatmap (much easier to read)
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.tight_layout()
plt.show()

In [ ]:
#c)	Remove the 3 variables with the highest p-values. Refit the model
#   How have the p-values for the other variables changed? Did R2 change by much?

import statsmodels.api as sm

#define outcome and predictors
X = smokes_df[['Age', 'HS', 'Income', 'Black', 'Female', 'Price']]
y = smokes_df['Sales']

#add constant
X = sm.add_constant(X)

#initial model
model_full = sm.OLS(y, X).fit()
print("Initial Model Summary:")
print(model_full.summary())

#find the 3 variables with highest p-values (excluding the constant)
pvals = model_full.pvalues.drop('const')
highest_pvars = pvals.sort_values(ascending=False).head(3).index.tolist()

#drop those 3 predictors
X_reduced = X.drop(columns=highest_pvars)

#refit model
model_reduced = sm.OLS(y, X_reduced).fit()
print("\nRefit Model After Removing Highest P-Value Predictors:")
print(model_reduced.summary())

R² dropped slightly: from 0.321 (full model) to 0.303 (reduced model), which means the reduced model explains about 2% less of the variance in cigarette sales (which is acceptable)

Adjusted R² increasedfrom 0.228 (full model) to 0.259 (reduced model), suggesting the reduced model is more efficient

In [ ]:
#how have the p-values changed

#create a side-by-side comparison DataFrame
pvals_comparison = pd.DataFrame({
    "Full_Model_p": model_full.pvalues,
    "Reduced_Model_p": model_reduced.pvalues
})

print(pvals_comparison)

Variable	Full Model p-value	Reduced Model p-value	Change
Age	        0.167	            0.065	                ↓ More significant
Black	    0.467		        Removed                 (not significant)
Female	    0.851		        Removed                 (not significant)
HS	        0.940		        Removed                 (not significant)
Income	    0.070	            0.007	                ↓ More significant
Price	    0.003	            0.001	                ↓ Slightly more significant
const	    0.676	            0.305	                ↓ Still not significant




a)	What does R2 tell you about the fit of the second model?
    
    •	Reduced model R² (0.303) means that 30.3% of the variation in cigarette sales across states is explained by the predictors in the reduced model (Age, Income, and Price).
    
	•	This tells us that the model captures some structure in the data, yet nearly 70% of the variation remains unexplained — suggesting that other non-measured factors may likely influence cigarette sale, too.


Compared to the Full Model:
	•	The full model had R² = 0.321, so removing variables caused only a small drop in explanatory power (about 1.8%).
    
	•	But the adjusted R² actually increased from 0.228 → 0.259, which suggests the  REDUCED MODEL IS MORE EFFICIENT by explaining almost the same amount of variation with fewer, more meaningful predictors.


The reduced model, while not perfect, is statistically meaningful and simpler, which improves model quality.

Lower AIC values are better, because they indicate a model that fits the data well without unnecessary complexity (the balance of fit and complexity).  Lower AIC values indicate a better balance.  In our case the Full model AIC is 491.7 and the Reduced model AIC is 487.

Even though the Reduced model uses fewer predictors, it performed better overall when accounting for complexity.

In [ ]:
#3)	Outliers
#a)	Do a leverage plot. Are there influential outliers? 

#influence plot
fig, ax = plt.subplots(figsize=(10, 6))
sm.graphics.influence_plot(model_reduced, ax=ax, criterion="cooks")

plt.title("Influence Plot (Leverage vs. Studentized Residuals)")
plt.grid(True)
plt.show()


Are there influential outliers?

Yes, I see a large bubble in the top left.  This is a high-leverage, high-residual point.  This an influential outlier and may disproportionately affect my model.

In [ ]:
#4)	Linearity & constant variance
#   a)	Generate a predicted vs standardized residual plot.
#       Does the data meet the linearity assumption?

#import numpy as np
#import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence

#compute influence
influence = OLSInfluence(model_reduced)

#extract fitted values and standardized residuals
fitted_vals = model_reduced.fittedvalues
standardized_residuals = influence.resid_studentized_internal

#Predicted vs Standardized Residuals plot
plt.figure(figsize=(10, 6))
plt.axhline(y=0, color='gray', linestyle='--', linewidth=1)
plt.scatter(fitted_vals, standardized_residuals, alpha=0.7)
plt.title("Predicted vs. Standardized Residuals")
plt.xlabel("Predicted Values")
plt.ylabel("Standardized Residuals")
plt.grid(True)
plt.show()

Does the data meet the linearity assumption?

The data seems to reasonably meet the linearity assumption (by visual observation).

The residuals are scattered fairly randomly around the horizontal zero line. I see no "strong" curve or clear pattern.  If so, I'd lean toward non-linearity.  Furthermore, most of the standardized residuals fall within plus/minus 2, which I understand is common and expected.  I don't believe the few (moderate) outliers are enough to indicate a loss of linearity.


In [ ]:
#5)	Normality
#   a)	Do a Q-Q Plot of the residuals.
#   Are the residuals normally distributed?


#get the residuals
residuals = model_reduced.resid

# Generate the Q-Q plot
sm.qqplot(residuals, line='45', fit=True)
plt.title("Q-Q Plot of Residuals")
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Sample Quantiles")
plt.grid(True)
plt.show()

Are the residuals normally distributed?

Well, mostly, they are normally distributed.  At the tail, we do see a couple of outliers (an indication of positive skew).  It's seemingly, normally distributed in the central portion.

With this data, the outliers could represent a problem/challenge to the normality of the residuals.  This is because in regression, a general rule of thumb is to have at least 10–15 observations per predictor to ensure stability in estimates. We may not have enough here.  I understand there are options for further cleansing and clarification, which I have not done here. 